In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

In [3]:
def df_movies():
    title_basics1 = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep ='\t', low_memory = False)
    title_basics1 = title_basics1.replace('\\N', np.NaN)
    title_basics1 = title_basics1[title_basics1['isAdult'] == '0']
    title_basics1['runtimeMinutes'] = title_basics1['runtimeMinutes'].replace('Reality-TV', np.NaN)
    title_basics1['runtimeMinutes'] = title_basics1['runtimeMinutes'].replace('Documentary', np.NaN)
    title_basics1['runtimeMinutes'] = title_basics1['runtimeMinutes'].replace('Talk-Show', np.NaN)
    title_basics1['runtimeMinutes'] = title_basics1['runtimeMinutes'].replace('Game-Show', np.NaN)
    title_basics1['runtimeMinutes'] = title_basics1['runtimeMinutes'].replace('Animation,Comedy,Family', np.NaN)
    title_basics1['runtimeMinutes'] = title_basics1['runtimeMinutes'].astype(float)
    
    movies_genre = title_basics1.copy()
    df_genre2 = title_basics1['genres'].str.split(',', expand=True)
    movies_genre['genre 1'] = df_genre2[0]
    movies_genre['genre 2'] = df_genre2[1]
    movies_genre['genre 3'] = df_genre2[2]
    movies_genre = movies_genre[movies_genre['titleType'] =='movie']
    return movies_genre

movies = df_movies()

In [4]:
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genre 1,genre 2,genre 3
498,tt0000502,movie,Bohemios,Bohemios,0,1905,NaN,100.0,NaN,NaN,NaN,NaN
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70.0,"Action,Adventure,Biography",Action,Adventure,Biography
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90.0,Drama,Drama,None,None
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,NaN,NaN,Drama,Drama,None,None
625,tt0000630,movie,Hamlet,Amleto,0,1908,NaN,NaN,Drama,Drama,None,None


In [5]:
movies.shape

(592276, 12)

In [6]:
def movie_rating():
    title_ratings= pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep = '\t')
    movie_rating = movies.merge(title_ratings, on='tconst', how='inner')
    return movie_rating 
    
movie_rating = movie_rating()

In [32]:
movies_rating_dropna_year = movie_rating[movie_rating['startYear'].notna()].copy()
movies_rating_dropna_year['startYear'] = movies_rating_dropna_year['startYear'].astype(int)

In [8]:
def decennie(x):
    if x < 1900:
        return '<1900'
    elif 1900 <= x < 1910:
        return '1900 - 1910'
    elif 1910 <= x < 1920:
        return "1910 - 1920"
    elif 1920 <= x < 1930:
        return "1920 - 1930"
    elif 1930 <= x < 1940:
        return "1930 - 1940"
    elif 1940 <= x < 1950:
        return "1940 - 1950"
    elif 1950 <= x < 1960:
        return "1950 - 1960"
    elif 1960 <= x < 1970:
        return "1960 - 1970"
    elif 1970 <= x < 1980:
        return "1970 - 1980"
    elif 1980<= x < 1990:
        return "1980 - 1990"
    elif 1990 <= x < 2000:
        return "1990 - 2000"
    elif 2000 <= x < 2010:
        return "2000 - 2010"
    elif 2010 <= x < 2020:
        return "2010 - 2020"
    elif 2020 <= x < 2030:
        return "2020 - 2030"

In [40]:
movies_rating_dropna_year['decade'] = movies_rating_dropna_year['startYear'].apply(decennie)
pivot_top100 = movie_rating.pivot_table(index=['primaryTitle', 'tconst'], values = ['averageRating', 'numVotes'])
top100 = pivot_top100.sort_values(['numVotes'], ascending=[False])[:100]
top100_df = pd.DataFrame(top100)

In [34]:
movies_rating_dropna_year.shape

(269998, 15)

In [16]:
movies_rating_dropna_year.isna().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
endYear           269998
runtimeMinutes     26980
genres             10152
genre 1            10152
genre 2           137287
genre 3           208495
averageRating          0
numVotes               0
decade                 0
dtype: int64

In [42]:
final_movie_rating = movies_rating_dropna_year.copy()
final_movie_rating = final_movie_rating[['decade', 'runtimeMinutes', 'startYear', 'genre 1']]
final_movie_rating.head()

,decade,runtimeMinutes,startYear,genre 1
0,1900 - 1910,100.0,1905,NaN
1,1900 - 1910,70.0,1906,Action
2,1900 - 1910,90.0,1907,Drama
3,1900 - 1910,NaN,1907,Drama
4,1900 - 1910,NaN,1908,Drama


In [43]:
final_movie_rating.dropna(inplace=True)

In [44]:
final_movie_rating.reset_index(inplace=True, drop=True)

In [45]:
final_movie_rating.to_csv('../data/final_movie_rating.csv.zip', compression='zip')

decade 
runtime > 60
startyear

In [10]:
def get_actors():
    principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep ='\t', usecols=['tconst', 'nconst', 'category'])
    actors_df = principals[(principals['category'] == 'actor')|(principals['category'] == 'actress')]
    names = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep ='\t')
    actors = actors_df.merge(names, how = 'inner', on = 'nconst')
    actors.drop(['primaryProfession', 'knownForTitles'], axis = 1, inplace = True)
    actors = actors.replace('\\N', np.NaN)
    actors['category'] = actors['category'].astype('category')
    actors_clean = actors.copy().dropna(subset=['birthYear'])
    actors_sans_death = actors_clean[actors_clean['deathYear'].isna()]
    living_actors = actors_sans_death.copy()
    living_actors.drop(columns=['deathYear'], inplace=True)
    living_actors['birthYear'] = living_actors['birthYear'].astype(int)
    living_actors.drop(living_actors[living_actors['birthYear'] == 21].index, inplace=True)
    def age(birthY):
        today = date.today()
        return today.year - birthY
    living_actors['age'] = living_actors['birthYear'].apply(age)
    living_actors.drop(living_actors[living_actors['age'] > 120].index, inplace=True)
    def age_period(age):
        if age < 18:
            return '<18'
        elif 18 <=age < 30:
            return '18-30'
        elif 30 <=age < 40:
            return '30-40'
        elif 40 <=age < 50:
            return '40-50'
        elif 50 <= age < 60:
            return '50-60'
        elif 60 <=age < 70:
            return '60-70'
        elif age >= 70:
            return '> 70'
    living_actors['age_fork'] = living_actors['age'].apply(age_period)
    top = top100_df.merge(living_actors, how = 'inner', on = 'tconst')
    return top

actors = get_actors()
top100_df_bis = movies_rating_dropna_year.sort_values(['numVotes', 'averageRating'], ascending = False)[:100]
top_100_actors = top100_df_bis.merge(actors, how='inner', on = 'tconst')

In [46]:
top_100 = top100_df_bis.copy()
top_100 = top_100[['tconst','numVotes', 'genre 1', 'averageRating', 'decade']]

In [48]:
top_100.to_csv('../data/top_100.csv.zip', compression='zip')

In [49]:
top_100_actors = top_100.merge(actors, how='inner', on = 'tconst')

In [50]:
top_10_actors = top_100_actors['primaryName'].value_counts()[:10]

In [52]:
top_10_actors.to_csv('../data/top_10_actors.csv.zip', compression='zip')

In [73]:
top_10_actors.index

Index(['Robert Downey Jr.', 'Leonardo DiCaprio', 'Tom Hanks', 'Chris Evans',
       'Matt Damon', 'Orlando Bloom', 'Christian Bale', 'Natalie Portman',
       'Mark Ruffalo', 'Ian McKellen'],
      dtype='object')

In [76]:
top_10_actors.values

array([7, 7, 6, 6, 4, 4, 4, 4, 4, 4], dtype=int64)

In [54]:
actors_df = actors[['age_fork', 'category']]

In [56]:
actors_df.to_csv('../data/actors_df.csv.zip', compression='zip')

In [79]:
test = pd.read_csv('../data/top_10_actors.csv.zip', index_col=0)

In [80]:
test

,primaryName
Robert Downey Jr.,7
Leonardo DiCaprio,7
Tom Hanks,6
Chris Evans,6
Matt Damon,4
Orlando Bloom,4
Christian Bale,4
Natalie Portman,4
Mark Ruffalo,4
Ian McKellen,4


In [82]:
top_10_actors.index

Index(['Robert Downey Jr.', 'Leonardo DiCaprio', 'Tom Hanks', 'Chris Evans',
       'Matt Damon', 'Orlando Bloom', 'Christian Bale', 'Natalie Portman',
       'Mark Ruffalo', 'Ian McKellen'],
      dtype='object')

In [11]:
def get_real():
    principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep ='\t', usecols=['tconst', 'nconst', 'category'])
    real_df = principals[principals['category'] == 'director']
    top_real = top100_df_bis.merge(real_df, how='left', on = 'tconst')
    names = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep ='\t')
    top_real_df = top_real.merge(names, how = 'left', on = 'nconst')
    return top_real_df

top_real_df = get_real()

In [66]:
top_real = top_real_df[['primaryName']]

In [71]:
pd.read_csv('../data/top_real.csv.zip', index_col=0)

,primaryName
0,Frank Darabont
1,Christopher Nolan
2,Christopher Nolan
3,David Fincher
4,Robert Zemeckis
...,...
108,Ethan Coen
109,Jon Favreau
110,Sam Raimi
111,Richard Kelly


In [75]:
top_real['primaryName'].value_counts()[:10]

Christopher Nolan    7
Martin Scorsese      5
Quentin Tarantino    5
Steven Spielberg     5
James Cameron        4
Peter Jackson        4
Ridley Scott         3
Anthony Russo        3
Joe Russo            3
Lee Unkrich          3
Name: primaryName, dtype: int64

In [68]:
top_real.to_csv('../data/top_real.csv.zip', compression='zip')

In [61]:
top_real = pd.DataFrame(top_real_df['primaryName'].value_counts()[:10], names = ['actor_name', 'count'])

TypeError: __init__() got an unexpected keyword argument 'names'

In [60]:
top_real

,primaryName
Christopher Nolan,7
Martin Scorsese,5
Quentin Tarantino,5
Steven Spielberg,5
James Cameron,4
Peter Jackson,4
Ridley Scott,3
Anthony Russo,3
Joe Russo,3
Lee Unkrich,3


In [87]:
df_algo = pd.read_csv('../df_algo.csv', index_col=0)

In [89]:
df_algo[df_algo['startYear'] > 1990]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,...,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
1595,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",6.4,81868,...,0,0,0,1,0,0,0,0,0,0
6240,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama,6.8,6913,...,0,0,0,0,0,0,0,0,0,0
9198,tt0095111,movie,Escape,El heroob,0,1991,122,"Action,Crime,Drama",7.7,846,...,0,0,0,0,0,0,0,0,0,0
9411,tt0097416,movie,Kadhalan,Kadhalan,0,1994,168,"Action,Drama,Romance",7.2,1919,...,0,0,0,1,0,0,0,0,0,0
9596,tt0099558,movie,Operation Condor,Fei ying gai wak,0,1991,106,"Action,Adventure,Comedy",7.3,19834,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30315,tt9905412,movie,Ottam,Ottam,0,2019,120,Drama,7.2,563,...,0,0,0,0,0,0,0,0,0,0
30316,tt9905462,movie,Pengalila,Pengalila,0,2019,111,Drama,8.2,671,...,0,0,0,0,0,0,0,0,0,0
30317,tt9906644,movie,Manoharam,Manoharam,0,2019,122,"Comedy,Drama",6.8,703,...,0,0,0,0,0,0,0,0,0,0
30318,tt9911196,movie,The Marriage Escape,De beentjes van Sint-Hildegard,0,2020,103,"Comedy,Drama",7.6,3215,...,0,0,0,0,0,0,0,0,0,0


In [94]:
df_algo = df_algo[(df_algo['startYear'] >= 1980) & (df_algo['startYear'] <= 2021)]

In [96]:
df_algo =df_algo[(df_algo['runtimeMinutes'] >= 60) & (df_algo['runtimeMinutes'] <= 240)]

In [100]:
df_algo = df_algo[df_algo['averageRating'] >= 6.0]

In [102]:
df_algo[df_algo['numVotes'] >= 2000]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,...,Musical,Mystery,News,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
1595,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,118,"Comedy,Fantasy,Romance",6.4,81868,...,0,0,0,1,0,0,0,0,0,0
5877,tt0066498,movie,The Ear,Ucho,0,1990,94,"Drama,Thriller",7.8,2743,...,0,0,0,0,0,0,0,1,0,0
6046,tt0067625,movie,Trial on the Road,Proverka na dorogakh,0,1986,96,"Drama,War",7.9,2172,...,0,0,0,0,0,0,0,0,1,0
6240,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,122,Drama,6.8,6913,...,0,0,0,0,0,0,0,0,0,0
7127,tt0076276,movie,Who's Singin' Over There?,Ko to tamo peva,0,1980,86,"Adventure,Comedy,Drama",8.8,14895,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30308,tt9893250,movie,I Care a Lot,I Care a Lot,0,2020,118,"Comedy,Crime,Thriller",6.3,117218,...,0,0,0,0,0,0,0,1,0,0
30313,tt9900782,movie,Kaithi,Kaithi,0,2019,145,"Action,Crime,Thriller",8.5,18415,...,0,0,0,0,0,0,0,1,0,0
30314,tt9902160,movie,Herself,Herself,0,2020,97,Drama,7.0,2831,...,0,0,0,0,0,0,0,0,0,0
30318,tt9911196,movie,The Marriage Escape,De beentjes van Sint-Hildegard,0,2020,103,"Comedy,Drama",7.6,3215,...,0,0,0,0,0,0,0,0,0,0


In [104]:
df_algo.to_csv('../df_algo2.csv')